# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [7]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 28

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  28  
1 :  Piston is a two-rowed spring feed barley.
2 :  It was developed by CENEX, Oregon.
3 :  It is medium maturing (heads about 7 days later than Steptoe) and has medium plant height (2 inches shorter than Steptoe) and good straw strength.
4 :  At the time of evaluation it was moderately resistant to leaf rust and susceptible to scald and BYD.
5 :  It was evaluated as Entry 698 in the UC Regional Cereal Testing program from 1985-1987 for spring planting in the intermountain area of northern California and for late fall planting in the Central Valley and the south-central coastal regions of California.    
6 :  POCO   
7 :  Poco is a six-rowed spring feed barley.
8 :  It was developed by D.G. Lorance for Anderson Clayton Co., and released in 1981.
9 :  Poco was selected from an early maturing CIMMYT line from the cross HJA
10 :  C4715*Olli-
11 :  M64-69.
12 :  Its experimental designations were CMB73-383-2Y-2B-Y-0B and PC238.
13 :  Poco has very early maturity, very short statu

## Read in Per-line named entity file and match entities to sentence positions.

In [10]:
import re
import csv
import pandas as pd
fname = "Data/barley_p28_ner.txt"

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


Handle manually:  ['9', 'HJA C4715*Olli- M64-69', 'PED']
Handle manually:  ['26', 'CM 67/3*Briggs/4/Briggs*4/3/California Mariout*4/CIho 1179//2*California Mariout*6/Club Mariout', 'PED']
Handle manually:  ['39', 'Crop Science 19:741 (1979)', 'JRNL']
Handle manually:  ['56', '(Induced mutant in Harrington) ant29-667/Baronesse', 'PED']
Handle manually:  ['69', 'Crop Science 44:1859-1860 (2004)', 'JRNL']
    Index                                           Sentence  \
0       1          Piston is a two-rowed spring feed barley.   
1       1          Piston is a two-rowed spring feed barley.   
2       1          Piston is a two-rowed spring feed barley.   
3       1          Piston is a two-rowed spring feed barley.   
4       1          Piston is a two-rowed spring feed barley.   
..    ...                                                ...   
142    67  At the time of release it  was moderately resi...   
143    67  At the time of release it  was moderately resi...   
144    67  At the 

## Create a function to clean up overlapping intervals

In [11]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [12]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
for i in range(0,len(records)):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]}),")
    #print("    ('"+records[i]['Sentence']+"', {'entities': ["+records[i]['MatchInfo']+"]}),")


    ('Piston is a two-rowed spring feed barley.', {'entities': [(0, 6, 'CVAR'), (12, 21, 'TRAT'), (22, 28, 'TRAT'), (29, 33, 'TRAT'), (34, 40, 'CROP')]}),
    ('Its experimental designations were CMB73-383-2Y-2B-Y-0B and PC238.', {'entities': [(35, 55, 'ALAS'), (60, 65, 'ALAS')]}),
    ('Poco has very early maturity, very short stature, and good straw strength.', {'entities': [(0, 4, 'CVAR'), (20, 28, 'TRAT'), (41, 48, 'TRAT'), (59, 73, 'TRAT')]}),
    ('The spike is parallel, mid-dense, and inclined at maturity.', {'entities': [(4, 9, 'PLAN')]}),
    ('The basal rachis internode is curved and the rachis edges are covered with hairs.', {'entities': [(4, 26, 'PLAN'), (45, 57, 'PLAN'), (75, 80, 'PLAN')]}),
    ('The lemma awn is long and rough.', {'entities': [(4, 13, 'PLAN')]}),
    ('The glume awn is longer than the glume and rough.', {'entities': [(4, 13, 'PLAN'), (33, 38, 'PLAN')]}),
    ('The glume is one-half the length of the lemma.', {'entities': [(4, 9, 'PLAN'), (40, 45, 'PLAN')